# Calculate downhole xyz from Dist,Azimuth,Inclination using minimum curvature

<img src='http://www.drillingformulas.com/wp-content/uploads/2010/08/Minimum-Curvature-Method.jpg'>

The Minimum Curvature Method smooths two straight-line segments of the Balanced Tangential Method by using the Ratio Factor (RF).

Where:  
MD = Measured Depth between surveys  
I1 = Inclination (angle) of upper survey in degrees  
I2 = Inclination (angle) of lower in degrees  
Az1= Azimuth direction of upper survey  
Az2 = Azimuth direction of lower survey  
RF = Ratio Factor  
ß  is the dog leg angle.  

Thanks to:

http://www.drillingformulas.com/minimum-curvature-method/ 

and 

https://gis.stackexchange.com/questions/13484/how-to-convert-distance-azimuth-dip-to-xyz

In [14]:
from math import acos, cos, sin, tan, radians

def dia2xyz(X1,Y1,Z1,I1,Az1,Distance1,I2,Az2,Distance2):
    I1=radians(I1)
    Az1=radians(Az1)
    I2=radians(I2)
    Az2=radians(Az2)

    MD = Distance2 - Distance1

    Beta = acos(cos(I2 - I1) - (sin(I1)*sin(I2)*(1-cos(Az2-Az1))))
    if(Beta==0):
        RF=1
    else:
        RF = 2 / Beta * tan(Beta / 2)

    dX = MD/2 * (sin(I1)*sin(Az1) + sin(I2)*sin(Az2))*RF
    dY = MD/2 * (sin(I1)*cos(Az1) + sin(I2)*cos(Az2))*RF
    dZ = MD/2 * (cos(I1) + cos(I2))*RF

    X2 = X1 + dX
    Y2 = Y1 + dY
    Z2 = Z1 - dZ
    
    return(X2,Y2,Z2)

In [23]:
X1=Y1=Z1=0

Distance1=3500 # start
I1=15 # Inclination start
Az1=20 # Azimuth start

Distance2=3600 # end
I2=25 # Inclination end
Az2=45 # Azimuth end

X2,Y2,Z2=dia2xyz(X1,Y1,Z1,I1,Az1,Distance1,I2,Az2,Distance2)

print('E,N,Z')
print(X1,Y1,Z1)
print(X2,Y2,Z2)

E,N,Z
0 0 0
19.45077374667729 27.218328569754362 -94.01234323216109


In [16]:
import pandas as pd

survey=pd.read_csv('../../drilling_test/a104970_a104970_drilling/Survey_data_file.txt', skiprows=47,sep='\t')
survey.drop(survey.index[[0,1]],inplace=True)
survey.drop(survey.index[len(survey)-1],inplace=True)
survey.reset_index(inplace=True)
display(survey.head())

location=pd.read_csv('../../drilling_test/a104970_a104970_drilling/Location_data_file.txt', skiprows=45,sep='\t')
location.drop(location.index[[0,1]],inplace=True)
location.drop(location.index[len(location)-1],inplace=True)
location.reset_index(inplace=True)
location.set_index('Hole_ID',inplace=True)

display(location.head())


,index,H1000,Hole_ID,Surveyed Depth,Dip,Azimuth_TRUE,Survey Instrument,Surv_Company,Drill Code,Projectcode,SURVGRIDNAME_DT_SRC,AZIMUTH_DT_SRC,AZIMUTHGRIDNAME,SurvComments,MagAzimuth,MagCorrection,MagStrength
0,2,D,MWD14001,0,-60,210,PL,NaN,NaN,RDEX,MGA_Z51S,210.0,MGA_Z51S,NaN,208.0,2.0,NaN
1,3,D,MWD14001,21,-60.4,210.3,REFLEX_SS,NaN,NaN,RDEX,MGA_Z51S,210.3,MGA_Z51S,NaN,208.3,2.0,NaN
2,4,D,MWD14001,32.2,-60.6,209.6,REFLEX_SS,NaN,NaN,RDEX,MGA_Z51S,209.6,MGA_Z51S,NaN,207.6,2.0,NaN
3,5,D,MWD14001,66,-60.5,207.9,REFLEX_MS,NaN,NaN,RDEX,MGA_Z51S,207.9,MGA_Z51S,NaN,205.9,2.0,NaN
4,6,D,MWD14001,72,-60.4,208.1,REFLEX_MS,NaN,NaN,RDEX,MGA_Z51S,208.1,MGA_Z51S,NaN,206.1,2.0,NaN


,index,H1000,MGA Easting,MGA Northing,Elevation,Pre_Collar_Depth,Total Hole Depth,Drill Code,Dip,Azimuth_TRUE,...,THOMSONS_PL_Z,THOMSONS_S_X,THOMSONS_S_Y,THOMSONS_S_Z,TROTMANS_PL_X,TROTMANS_PL_Y,TROTMANS_PL_Z,TROTMANS_S_X,TROTMANS_S_Y,TROTMANS_S_Z
Hole_ID,,,,,,,,,,,,,,,,,,,,,
MWD14001,2,D,420324,7584887,314,NaN,648.7,NaN,-60,210,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MWD14002,3,D,421814,7583639,314,NaN,528.5,NaN,-60,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
def convert_survey(survey,location):
    holeid=''
    for indx,interval in survey.iterrows():
        if(interval['Hole_ID'] != holeid):
            first=True
            
        if(first):
            first=False
            lasti=float(interval['Dip'])
            lasta=float(interval['Azimuth_TRUE'])
            lastd=float(interval['Surveyed Depth'])
            holeid=interval['Hole_ID']
            X1=float(location.loc[holeid]['MGA Easting'])
            Y1=float(location.loc[holeid]['MGA Northing'])
            Z1=float(location.loc[holeid]['Elevation'])
            print()
            print(holeid,indx+2,X1,Y1,Z1)
        else:
            if(indx<len(survey) and interval['Hole_ID'] == holeid):
                X2,Y2,Z2=dia2xyz(X1,Y1,Z1,lasti,lasta,lastd,float(interval['Dip']),float(interval['Azimuth_TRUE']),float(interval['Surveyed Depth']))
                print(holeid,interval['index'],X2,Y2,Z2)
                X1=X2
                Y1=Y2
                Z1=Z2
                lasti=float(interval['Dip'])
                lasta=float(interval['Azimuth_TRUE'])
                lastd=float(interval['Surveyed Depth'])    
                
convert_survey(survey,location)



MWD14001 2 420324.0 7584887.0 314.0
MWD14001 3 420333.1528705034 7584902.75763067 303.5635500581167
MWD14001 4 420338.0193951451 7584911.20382948 298.0483570185162
MWD14001 5 420352.1755308132 7584937.006610235 281.42919671408026
MWD14001 6 420354.625956783 7584941.615200808 278.4700974090309
MWD14001 7 420357.08199202496 7584946.214946074 275.5018950505092
MWD14001 8 420359.54081724666 7584950.810260398 272.52914247053315
MWD14001 9 420362.0156643821 7584955.39695688 269.5563853354977
MWD14001 10 420364.49173617474 7584959.98593733 266.5881778552873
MWD14001 11 420366.9545698436 7584964.579102712 263.6154273539461
MWD14001 12 420369.40971662744 7584969.167537559 260.62904092323237
MWD14001 13 420371.8793472038 7584973.754081886 257.6517361872421
MWD14001 14 420374.37018834933 7584978.332118366 254.67898511651987
MWD14001 15 420376.857308361 7584982.903301377 251.69259868580613
MWD14001 16 420379.3170361326 7584987.471414774 248.67900589726722
MWD14001 17 420381.77444510453 7584992.02